# Tutorial for HPS multiple DDX4 

In [5]:
# load packages
import numpy as np
import pandas as pd
import sys
import os
import simtk.openmm as mm
import simtk.openmm.app as app
import simtk.unit as unit

sys.path.append('../../')
from openabc.forcefields.parsers import HPSParser
from openabc.forcefields import HPSModel
from openabc.utils.helper_functions import build_straight_CA_chain, write_pdb

# set simulation platform
platform_name = 'CPU'

Parse a single DDX4.

In [6]:
sequence = 'MGDEDWEAEINPHMSSYVPIFEKDRYSGENGDNFNRTPASSSEMDDGPSRRDHFMKSGFASGRNFGNRDAGECNKRDNTSTMGGFGVGKSFGNRGFSNSRFEDGDSSGFWRESSNDCEDNPTRNRGFSKRGGYRDGNNSEASGPYRRGGRGSFRGCRGGFGLGSPNNDLDPDECMQRTGGLFGSRRPVLSGTGNGDTSQSRSGSGSERGGYKGLNEEVITGSGKNSWKSEAEGGES'
ca_pdb = 'init_DDX4_CA.pdb'
ca_atoms = build_straight_CA_chain(sequence, r0=0.38)
write_pdb(ca_atoms, ca_pdb)
protein_parser = HPSParser(ca_pdb)

Parse configuration with default settings.


Prepare the initial configuration with gmx insert-molecules. 

In [7]:
# use gmx insert-molecules to put molecules into the simulation box randomly
n_mol = 10
cmd = f'gmx insert-molecules -ci {ca_pdb} -nmol {n_mol} -box 50 50 50 -radius 1.0 -scale 2.0 -o start.pdb'
if not os.path.exists('start.pdb'):
    os.system(cmd)

Use the Urry scale optimal parameter ($\mu=1$ and $\Delta=0.08$) and run the simulation. 

In [8]:
protein = HPSModel()
for i in range(n_mol):
    protein.append_mol(protein_parser)
top = app.PDBFile('start.pdb').getTopology()
init_coord = app.PDBFile('start.pdb').getPositions()
protein.create_system(top)
protein.add_protein_bonds(force_group=1)
protein.add_contacts('Urry', mu=1, delta=0.08, force_group=2)
protein.add_dh_elec(force_group=3)
temperature = 300*unit.kelvin
friction_coeff = 0.01/unit.picosecond # use smaller friction coefficient to accelerate dynamics
timestep = 10*unit.femtosecond
integrator = mm.LangevinMiddleIntegrator(temperature, friction_coeff, timestep)
platform_name = 'CPU'
protein.set_simulation(integrator, platform_name, init_coord=init_coord)
protein.simulation.minimizeEnergy()
output_interval = 100
output_dcd = 'output_multi_DDX4.dcd'
protein.add_reporters(output_interval, output_dcd)
protein.simulation.context.setVelocitiesToTemperature(temperature)
protein.simulation.step(500)

Add protein bonds.
Set all the protein bond r0 values as 0.38
Set all the protein bond k_bond values as 8368.0
Add nonbonded contacts.
Use Urry hydropathy scale.
Scale factor mu = 1 and shift delta = 0.08.
Add Debye-Huckel electrostatic interactions.
Set Debye length as 1 nm.
Set water dielectric as 80.0.
Use platform: CPU
#"Step","Time (ps)","Potential Energy (kJ/mole)","Kinetic Energy (kJ/mole)","Total Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
100,1.0000000000000007,2464.33310346188,5895.802693256526,8360.135796718407,200.3962342193612,0
200,2.0000000000000013,3006.1597887156645,5400.610373128341,8406.770161844006,183.56482358183445,41.9
300,2.99999999999998,2947.3255796171024,5484.263100698315,8431.588680315417,186.40815000562654,42.4
400,3.9999999999999587,2808.8940578708634,5670.932403225793,8479.826461096656,192.7529731310082,42
500,4.999999999999938,2617.014443635183,5889.6126480703715,8506.627091705555,200.18583678757943,41.8
